# Single-voxel test

This notebook will show some of the functionality of a single voxel instances.

 1. The first example will examine a simple instance of a clump with only a few observed velocities. This example is fully worked-out with explanations for some of the steps hidden inside the voxel setup. It is compared to the analytical result to ensure accuracy, then repeated using more model velocities to show how the model becomes more accurate.
 2. Next it will be shown how absorption within the voxel is handled. A couple of examples will be given highlighting different scenarios where there are different numbers of clumps in a line-of-sight. These begin by evaluating the model, then the result is analytically approximated to show the accuracy.
 3. An example will be given to test the effect of the observed velocity dispersion on intensity.
 4. An example showcasing the ability of the voxel to be resized in order to fit a given number of the largest clump. It still uses a simple model with just one clump mass, but will test the effect on intensity when the number of clumps is normalised to different amounts.
 5. An example using multiple clump masses and showing how it affects the overall voxel emission. Here we repeat a previous example and just change which clump masses we use. Finally an example is given to show how a larger voxel with higher mass can produce the same intensity.

In [ ]:
from kosmatau3d import models
import numpy as np
from IPython.display import display, Latex

In [ ]:
np.set_printoptions(precision=8)

## Theoretical basis

We need to ensure that we can replicate the intensity and optical depth that we would expect from a $10 \ M_\odot$ clump in the voxel. In order to do that, let us consider the $\mathrm{C}^+ 1\! \rightarrow\! 0$ and $^{13}\mathrm{C}^+ 1\! \rightarrow\! 0$ transitions and the first dust wavelength (3.1 mm). In the radiative transfer equation,

\begin{align}
dI_\nu =& - I_\nu\, \kappa_\nu\, \mathrm{d}s + \epsilon_\nu\, \mathrm{d}s, & \\
I_\nu =& S_\nu\, (1-\mathrm{e}^{-\tau_\nu}) + I_\mathrm{bg, \nu}\, \mathrm{e}^{-\tau_\nu}, & (1) \\
\end{align}

where $S_\nu \equiv \frac{\epsilon_\nu}{\kappa_\nu}$, and we can assume $\kappa$ and $\epsilon$ (the absorption and emissivity coefficients, respectively) are constant in the voxel. These values are in principle a function of the observing velocity, $v_\mathrm{obs}$. Therefore they are calculated using the voxel-averaged emmision:

\begin{align}
\kappa_\nu (v_\mathrm{obs}) =& \frac{\left<\tau_\nu\right> (v_\mathrm{obs})}{\Delta s}, & (2) \\
\epsilon_\nu (v_\mathrm{obs}) =& \frac{\left<I_\nu\right> (v_\mathrm{obs})}{\Delta s}, & (3) \\
\end{align}

In order to properly understand these voxel-averaged values, it is necessary to have some insight for what is happening inside the voxel. The clumpy PDR is approximated by an ensemble of spherically-symetric KOSMA-$\tau$ 'clumps'. The ensemble of clumps has a Gaussian velocity distribution with dispersion $\sigma_\mathrm{ens}$, and each clump has an intrinsic velocity dispersion $\sigma_\mathrm{cl}$ (this value depends on which underlying KOSMA-$\tau$ grid is used, but currently it is $\sim0.71\, \frac{km}{s}$). The number of clumps with a particular mass ($N_j$) follows the mass spectrum:

\begin{align}
\frac{\mathrm{d}N}{\mathrm{d}M} =& A M^{-\alpha}. & (4) \\
\end{align}

Here we adopt $\alpha\! =\! 1.84$ from Heithausen et al. (1998), and $A$ is a constant. Henceforth the subscript $j$ will refer to a property specific for clumps with mass $M_j$. Clumps can be observed with a particular radial velocity, so we also use the formalism of the subscript $i$ to refer to a property with radial velocity $v_i$. Note that this is a velocity within the voxel, not the observed velocity. The observed radial velocity is $v_\mathrm{obs}$.

Now the number of clumps with a particular radial velocity $v_i$ is determined by the number of clumps of a particular mass $N_j$ and velocity distribution of the ensemble $\sigma_\mathrm{ens}$.

\begin{align}
\Delta N_{j,i} (v_\mathrm{obs}) =& \frac{N_j}{\sqrt{2 \pi}\, \sigma_\mathrm{ens}}\, \mathrm{exp} \left( -\frac{(v_\mathrm{vox}-v_i)^2}{2 \sigma_\mathrm{ens}} \right) \Delta v, & (5) \\
\end{align}

$p_{j,i}$ is the probability of having $k_{j,i}$ clumps in a line-of-sight observed at velocity $v_i$, determined by Binomial distribution of the number of clumps at that velocity:

\begin{align}
p_{j,i} =& \binom{N_{j,i}}{k_{j,i}} p_j^{k_{j,i}} (1-p_j)^{N_{j,i}-k_{j,i}}, & (6) \\
\end{align}

Where $p_j$ is the area fraction of clump $j$. If $p_j N_{j,i}>5$, then a gaussian expression is used with $\mu_{j,i} = p_j N_{j,i}$ and $\sigma_{j,i} = \sqrt{N_{j,i} p_j (1-p_j)}$.

For having $k_{j,i}$ clumps in a line-of-sight, the intensity and optical depth ($I_{\nu,j,i}$ \& $\tau_{\nu,j,i}$) can be calculated. Here the subscript $\nu$ means the properties refer to a particular transition (which is what we get from the KOSMA-$\tau$ grid). 

\begin{align}
I_{\nu,j,i} (v_{obs}) =& k_{j,i} I_{cl,j} \mathrm{exp} \left( \frac{(v_i-v_\mathrm{obs})^2}{2\sigma_\mathrm{cl}^2} \right), & (7) \\
\tau_{\nu,j,i} (v_{obs}) =& k_{j,i} \tau_{cl,j} \mathrm{exp} \left( \frac{(v_i-v_\mathrm{obs})^2}{2\sigma_\mathrm{cl}^2} \right). & (8) \\
\end{align}

Finally the intensity and optical depth can be averaged over the voxel using all possible combinations and their corresponding probabilities to determine voxel properties we require.

\begin{align}
\left< I_\nu \right> (v_\mathrm{obs}) =& \sum_i \left( \prod_j p_{j,i}(k_{j,i}) \right) \left( \sum_j k_{j,i} I_{\nu,j,i} (v_\mathrm{obs}) \right), & (9)\\
\left< \tau_\nu \right> (v_\mathrm{obs}) =& -\mathrm{ln} \left[ \sum_i \left( \prod_j p_{j,i}(k_{j,i}) \right) \ \mathrm{exp} \left( - \sum_j k_{j,i} \tau_{\nu,j,i} (v_\mathrm{obs}) \right) \right]. & (10) \\
\end{align}

Here the sums and products ensure we consider contributions from all clump masses at all radial velocities, since there is a velocity dispersion to each clump's intensity and optical depth. Now we can finally substitute these into the radiative transfer equation to determine the intensity we expect to observe from this voxel.

\begin{align}
I_{\mathrm{vox},\nu} (v_\mathrm{obs}) =& \frac{\epsilon_\nu (v_\mathrm{obs})}{\kappa_\nu (v_\mathrm{obs})} \left( 1 - e^{-\kappa_\nu (v_{obs}) \Delta s} \right) + I_\mathrm{bg} e^{-\kappa_\nu (v_\mathrm{obs}) \Delta s}. & (11) \\
\end{align}

## 1. Simple voxel

### Model result

We want to consider a small mass of clumps in a reasonably-large voxel. The instance can be initialised easily, but the voxel properties will require kwargs. The full list of kwargs can be seen in `Voxel.py`. We will define the kwargs as `parameters` for convenience, since it contains all of the arguments needed for setting the model and thus how the intensity and optical depth are calculated. At the moment this is a very long dictionary, so please let me know if you have any suggestions to make this simpler. It currently shows which kwargs affect the model and which kwargs set the actual voxel properties.

#### Grid Parameters

- *clump grids*
  - `clump_tau_grid_file`: The grid file containing clump-averaged optical 
  depth. 
  The default file, `clump_tau_LineCenter.dat`, has the dust optical depth grid 
  appended to the transition line optical depth grid.
  - `clump_tb_grid_file`: The grid file containing clump-averaged intensity. 
  The default file, `clump_Tmb_LineCenter.dat`, has the dust intensity grid 
  appended to the transition line intensity grid.
  - `clump_taufuv_grid_file`: The far-UV extinction grid.
  The default grid is `RhoMassAFUV.dat`.
  - `clump_column_density_file`: The mean column density grid.
  The default grid is `clumpMeanCols.dat`.
  - `clump_temperature_file`: The clump-averaged temperature grid.
  The default grid is `clumpTemperatures_filled.dat`
- *interclump grids*
  - `interclump_tau_grid_file`:  The grid file containing clump-averaged 
  transition line optical depth. 
  The default file is `interclumpTauLineCenter.dat`.
  - `interclump_tb_grid_file`: The grid file containing clump-averaged 
  transition line intensity. 
  The default file is `interclumpTmbLineCenter.dat`.
  - `interclump_dust_tau_grid_file`: The grid file containing clump-averaged 
  dust continuum optical depth. 
  The default file is `interclumpDustTau.dat`,
  - `interclump_dust_tb_grid_file`:  The grid file containing clump-averaged 
  dust continuum intensity. 
  The default file is `interclumpDustSED.dat`,
  - `interclump_taufuv_grid_file`: The far-UV extinction grid.
  The default grid is `interclumpTauFUV.dat`,
  - `interclump_column_density_file`: The mean column density grid.
  The default grid is `interclumpMeanCols.dat`,
  - `interclump_temperature_file`: The clump-averaged temperature grid.
  The default grid is `interclumpTemperatures_filled.dat`,

#### Model Parameters

- `voxel_size`: The size in parsecs of the voxel. The default is $1 pc$.
- `molecules`: This is a list of the species transitions included in the model. At the moment this is a string with '[molecule] [transition]'. The default is 'all' to include all species and transitions.
- `dust`: This is a list of dust wavelengths to include in the model. This must be set to specific values in `constants.dustNames`, 'molecular' for $50 \mu m < \lambda < 3.1 mm$, 'PAH' for $5 \mu m < \lambda < 3.1 mm$, or 'all' for all available wavelengths ($1 nm < \lambda < 3.1 mm$). Using more wavelengths increases computation time, and wavelengths less than $91 nm$ ($13.6 eV$) are not calculated.
- `alpha`: The exponent used in the mass spectrum power law for the clumps. The default is 1.84 as in Heithausen (1995).
- `gamma`: The exponent used in the mass-radius power law for the clumps. The default is 2.31 as in Heithausen (1995).
- `clumpMassRange`: KOSMA-$\tau^3$ accepts a list of clump types to be included in the model. The argument supplied here therefore is a list of clump mass ranges, specified in $dex(M_\odot)$. The values are interpolated from the KOSMA-$\tau$ grid included with this package, which covers the range $[-3, 3] \ dex$. By default this is [[0, 2], [-2]] to model the clump and diffuse interclump regions.
- `clumpMassNumber`: This specifies the number of different clump masses in the model. This should be a list with the length of the number of clump sets you want modelled. By default it is [3,1].
- `clumpNmax`: This is used for normalising the number of clumps that are calculated. This should be a list with the length of the number of clump sets you want modelled. If a float or int is given, that is the value used for each clump set. By default it is set to [1,100]
- `velocityRange`: The range of observing velocities in the model, in $\frac{km}{s}$. By default it is set to $[-10, 10]$. This does not affect the internal system velocities.
- `velocityNumber`: The number of observing velocities in `velocityRange`. By default it is set to $201$. This does not affect the internal system velocities.

#### Voxel Parameters

- `velocity`: The mean radial velocity of the clumps in the voxel in $\frac{km}{s}$. The default is $0 \frac{km}{s}$.
- `ensembleDispersion`: The velocity dispersion of the emissivity and absorption in the voxel. This should be a list with the length of the number of clump sets you want modelled. If a float or int is given, that is the value used for each clump set. The lower limit is the intrinsic velocity dispersion of each clump. The default is $[2, 5] \ \frac{km}{s}$.
- `volumeFactor`: The fraction of the voxel filled with the model clumps. Setting this kwarg will calculate the mean ensemble density from the `ensembleMass` and `voxel_size`, overriding any value given to the `ensembleDensity` kwarg.
- `ensembleMass`: The ensemble mass to be modeled in the voxel in $M_\odot$. This should be a list with the length of the number of clump sets you want modelled. If a float or int is given, that is the value used for each clump set. The default is $100 \ M_\odot$.
- `ensembleDensity`: The mean ensemble density in $cm^{-3}$ of the voxel, which is used to calculate the density of each clump. This should be a list with the length of the number of clump sets you want modelled. If a float or int is given, that is the value used for each clump set. These are interpolated from the KOSMA-$\tau$ grid included with this package, which covers the range $[10^3, 10^7] \ cm^{-3}$. The default is $[15000, 1911] \ cm^{-3}$.
- `FUV`: The FUV radiation in Draine units of the voxel. This should be a list with the length of the number of clump sets you want modelled. If a float or int is given, that is the value used for each clump set. These are interpolated from the KOSMA-$\tau$ grid included with this package, which covers the range $[10^0, 10^6] \ \chi_0$. The default is $[20000, 1] \ \chi_0$.
- `crir`: The cosmic ray ionisation rate per hydrogen atom, $\zeta_H$, which covers the range $[2 \times 10^{-16}, 10^{-13}] \ s^{-1}$. The default is the local value, $2 \times 10^{-16} s^{-1}$.

#### set properties

The voxel we want to make will only have one observing velocity and one clump mass so there is only one clump in the ensemble. For now we do not need more than one clump set, so the lists in model parameters for ensemble velocity dispersion, clump mass, ensemble density, and FUV radiation should all have length $1$. The ensemble density and FUV radiation of the voxel is $10^5 \ cm^{-3}$ and $10^4 \ \chi_0$, respectively. We want to create just one $10 \ M_\odot$ clump, so we use an ensemble velocity dispersion of $0 \ \frac{km}{s}$. After these values are set accordingly, unpack and pass the `parameters` disctionary to the `Voxel.setProperties` method.

In [ ]:
%%time

# Use these to change the voxel properties, which is necessary in the default mode.
parameters = {
              # Grid parameters
              'clump_tau_grid_file': 'clump_tau_LineCenter.dat',
              'clump_tb_grid_file': 'clump_Tmb_LineCenter.dat',
              'clump_taufuv_grid_file': 'RhoMassAFUV.dat',
              'clump_column_density_file': 'clumpMeanCols.dat',
              'clump_temperature_file': 'clumpTemperatures_filled.dat',
              'interclump_tau_grid_file': 'interclumpTauLineCenter.dat',
              'interclump_tb_grid_file': 'interclumpTmbLineCenter.dat',
              'interclump_dust_tau_grid_file': 'interclumpDustTau.dat',
              'interclump_dust_tb_grid_file': 'interclumpDustSED.dat',
              'interclump_taufuv_grid_file': 'interclumpTauFUV.dat',
              'interclump_column_density_file': 'interclumpMeanCols.dat',
              'interclump_temperature_file': 'interclumpTemperatures_filled.dat',

               # Model parameters
               'voxel_size' : 0.2,
               'transitions' : 'all',
               'abundances': ['ELECTR', 'H', 'H2', 'H+'],
               'dust' : 'PAH',
               'clump_mass_range' : [[1]],
               'clump_mass_number' : [1],
               'clump_n_max' : [1],
               'velocity_range' : [-10, 10],
               'velocity_number' : 101,
               'suggested_calc' : True,
               'interclump_idx': [False,],

               # Voxel properties
               'velocity' : 0.,
               'ensemble_dispersion' : 2,
               'ensemble_mass' : 1e1,
               'ensemble_density' : 1e5,
               'fuv' : 1e5,
               }

vox = models.Voxel()
vox.set_properties(**parameters)

Here we can verify that we are modelling an appropriate clump mass. These should be exactly the same since there is no velocity dispersion.

*Note:* This check is not functioning correctly with the new calculation.

In [ ]:
vox.get_ensemble_mass() #mass of the ensemble to be modelled in M_sol. This is merely the input given to vox.setProperties().

In [ ]:
vox.get_model_mass() #mass of the clumps that have been modelled in M_sol. This is the mass of the voxel represented in clumps, which may differ from the input due to discretisation.

Likewise for the ensemble-averaged density:

In [ ]:
vox.get_density()

We can also verify the volume-filling factor to ensure it is less that $1$. (i.e. The ensemble volume is less than the voxel volume.) We only have one clump in this example, so this should be the case.

In [ ]:
vox.get_volume_filling_factor()

That works fine. It is important to note that there are numerous things going on in this method, such as

 - setting the voxel properties (mass, density, FUV field),
 - reading the molecules available on the grid,
 - determining the intensity (in K) and optical depth from each clump in the model,
 - calculating how many clumps are at each observed radial velocity,
 - and calculating the clump combinations
 
We can check all of these directly from their corresponding sub-modules. To check the model parameters, examine the `constants` sub-module (so-called since it contains values constant in the model, not values that are always constant). We can verify our model clumps, observed velocities, and check which species transitions and dust wavelengths we can calculate. Note that the clump properties are lists since you can evaluate more than one set of clumps. We only evaluated one set, so we only need the first (only) element.

In [ ]:
models.constants.clump_log_mass[0] #in dex(M_sol)

In [ ]:
models.constants.velocity_range #in km/s

In [ ]:
models.species.clump_transitions#.reshape(-1,1)

In [ ]:
models.constants.dust_names

The `ensemble` sub-module takes the voxel properties and combines it with the model velocities to calculate the ensemble distribution, ie. how many clumps are observed at each velocity. This *binning* is described in Silke's thesis. If you have $N_j$ clumps of mass $j$, the number of clumps with velocity $i$ is,

$$
\Delta N_{j,i} = \frac{N_j}{\sqrt{2 \pi} \sigma_\mathrm{ens}} exp \left(-\frac{(v_\mathrm{vox}-v_i)^2}{2(\sigma_\mathrm{ens})^2}\right) \Delta v
$$

Here $\sigma_\mathrm{ens}$ is the velocity dispersion of the ensemble, $v_\mathrm{vox}$ is the radial velocity of the voxel (which is 0 for our test case), and $\Delta v$ is the velocity bin size. We can check the value of this quantity using one of the methods in `ensemble`.

In [ ]:
models.ensemble.print_ensembleparameters()

In [ ]:
models.ensemble.clumpNj

In [ ]:
models.ensemble.clumpDeltaNji

This method is already setup to display this information for each clump set (of which we have just one). Here you see why we chose a strange value for $\sigma_\mathrm{ens}$: at an observing velocity of $0 \frac{km}{s}$ this equation essentially becomes $\Delta N_{j} = N_j \Delta v$. We could have chosen $\sigma_\mathrm{ens}$ to be such that we only observe $1$ clump in the center velocity, but then we would have clumps at more than one observing velocity. That is fine, since we can clearly see one of the features of KOSMA-$\tau^3$: the normalisation of the ensemble. One of the kwargs we used in setting up the voxel was `clumpNmax`, which is used to scale the voxel to number of the largest clump. This is possible due to the relation between the voxel size and the number of clumps in the voxel. For $N_j$ clumps in a voxel of size $s$, we maintain that $N_j \propto s^2$. When the $\Delta N_{j,i}$ is normalised to $\Delta N_{\mathrm{norm}; \ j,i}$, this also affects the effective voxel size $s_\mathrm{eff}$, which in turn affects the probability of seeing a clump in the voxel. These effects combine to ensure the observed intensity is the same.

So in our example of $2$ clumps in a $1$ pc voxel are normalised to $1$ clump in a voxel with size $\frac{1}{\sqrt{2}}$ pc. When viewing the voxel face-on, the probability of seeing a clump along our line-of-sight is $\mathcal{P_j} = \frac{\pi R_j^2}{\Delta s^2}$, where $R_j$ is the clump radius assuming uniform density. This value can be obtained from the `masspoints` sub-module. It must always be smaller than the effective voxel size. We will see an example later on where the voxel has to be rescaled to fit the largest clump. What is most important to take away from this probability is the $\Delta s^{-1}$ dependance, so the intensity is kept constant since,

$$
\Delta N_{j,i} \mathcal{P_j} = \Delta N_{\mathrm{norm}; \ j,i} \mathcal{P_{\mathrm{eff}; \ j}}
$$

In [ ]:
models.masspoints.clump_radius[0] #in parsecs

In [ ]:
models.ensemble.clumpProbability[0]

The probabilities are used when calculating the FUV absorption, intensity, and optical depth for the different combinations of clumps that are visible in a line of sight. These actual combinations are defined in `ensemble` and used to setup the `combinations` sub-module. The latter sub-module is used to combine the combinations with the values determined in `masspoints`. In the setup of the voxel, this is used to determine the FUV absorption.

#### calculate emission

After you set the voxel properties, you can call the `Voxel.calculateEmission` method to finally calculate the voxel intensity and optical depth. The `verbose` kwarg is just to notify you when the emission has finished calculating.

In [ ]:
vox.calculate_emission(verbose=True)

The `masspoints` sub-module can be used to check the intensity and optical depth of each clump in the model. You can verify these values of each clump this way, as well as check the density and FUV field used to interpolate the KOSMA-$\tau$ grid.

In [ ]:
#i_trans

In [ ]:
transitions = ['C+ 1', 'C 1', 'CO 1']
i_trans = [np.where(np.asarray(models.species.clump_transitions)==trans)[0][0] for trans in transitions]
models.masspoints.clump_species_intensity[0][0, i_trans] #kn K; for the first dust line and the first transition of C+, C, and CO

In [ ]:
i_dust = models.constants.dust_names[models.constants.n_dust]=='3.1mm'
models.masspoints.clump_dust_intensity[0][0, i_dust]

In [ ]:
models.masspoints.clump_species_optical_depth[0][0, i_trans]

In [ ]:
models.masspoints.clump_dust_optical_depth[0][0, i_dust]

The intensity and optical depth of different clump combinations are attributes of `combinations`. These are lists with an array for each clump set. Each array has shape (len(combinations), len(clumps)). Our example has just one clump in the voxel, so the combination is [0, 1]. The lists of intensities and optical depths of each combination are accessed as follows,

In [ ]:
models.combinations.clump_species_intensity[0][:, i_trans]

In [ ]:
models.combinations.clump_dust_intensity[0][:, i_dust]

In [ ]:
models.combinations.clump_species_optical_depth[0][:, i_trans]

In [ ]:
models.combinations.clump_dust_optical_depth[0][:, i_dust]

Note how the first dimension of the matrices for line transmission are different
than that of the dust continuum.
This is due to how the probability is calculated: the line transition depends on
the maximum number of clumps in a velocity bin ($\mathrm{max}(\Delta N_{ji})$), 
whereas the dust continuum uses the total number of clumps ($\mathrm{max}(N_j)$).

#### plot results

Finally we can access the intensity and optical depth in the voxel using two methods: `getIntensity()` and `getOpticalDepth()`. There are also two methods to plot these properties in defferent ways. First there is `plotMolecule()`, which can be used to plot the intensity or optical depth of one or more molecules with respect to the radial velocity. The other is `plotSpectrum()`, which plots the intensity or optical depth of the dust continuum and identifies molecules by vertical lines, separating different molecules by colour.

For this example, we will first look at the spectrum at the central velocity. The properties we can view as a spectrum are the emissivity, absorption, and intensity after computing the radiative transfer equation. For this example, note that there is very little absorption and the emissivity and absorption are very similar.

In [ ]:
%matplotlib inline

vox.plot_spectrum(quantity='emissivity')

In [ ]:
vox.plot_spectrum(quantity='absorption')

In [ ]:
vox.plot_spectrum(quantity='intensity')

The molecular intensity and optical depth can be plotted against velocity, and this can be used to compare to the analytic result.

In [ ]:
vox.plot_transition(transition=['C+ 1', 'C 1', 'CO 1'], quantity='emissivity', logscale=False)

In [ ]:
vox.plot_transition(transition=['C+ 1', 'C 1', 'CO 1'], quantity='absorption', logscale=False)

In [ ]:
vox.plot_transition(transition=['C+ 1', 'C 1', 'CO 1'], quantity='intensity', logscale=False)

We can directly obtain the emission properties from the `Voxel` instance.

In [ ]:
transitions = ['C+ 1', 'C 1', 'CO 1']
i_trans = [np.where(np.asarray(models.species.clump_transitions)==trans)[0][0] for trans in transitions]

vox.get_species_emissivity(include_dust=False, total=True)[50, i_trans]

In [ ]:
i_dust = models.constants.dust_names[models.constants.n_dust]=='3.1mm'

vox.get_dust_emissivity(total=True)[50, i_dust]

In [ ]:
vox.get_species_absorption(include_dust=False, total=True)[50, i_trans]

In [ ]:
vox.get_dust_absorption(total=True)[50, i_dust]

In [ ]:
vox.get_species_intensity(include_dust=False, total=True)[50, i_trans]

In [ ]:
vox.get_dust_intensity(total=True)[50, i_dust]

In [ ]:
vox.get_taufuv()

### Analytic result

So if we have one $10 M_\odot$ clump in the voxel, we can calculate the voxel-averaged intensity and optical depth for the 3.1 mm dust continuum wavelength and the first transition of C+, C, and CO.
We should note that the clump radius is $0.0989\, \mathrm{pc}$, and the intensities and optical depths we consider are given in the following table.

|               | 3.1mm dust line | C+ 1         | C 1        | CO 1        |
| ------------- | --------------- | ------------ | ---------- | ----------- |
| intensity     | 0.0146 K        | 224 K        | 4.50 K     | 19.7 K      |
| optical depth | 0.0003          | 1.31         | 0.0517     | 0.491       |

We now use this in to calculate the radiative transfer equation:

\begin{align}
\epsilon_\mathrm{cl} =& T_\mathrm{B, cl}\, \frac{\tau_\mathrm{cl}}{\ell^\prime_\mathrm{cl}}\, (1 - \mathrm{e}^{-\tau_\mathrm{cl}}), \\
T_\mathrm{B, vox} =& \frac{\epsilon_\mathrm{cl}}{\kappa_\mathrm{cl}}\, 
\end{align}

where we use the effective length of each clump $\ell^\prime_\mathrm{cl} \equiv \frac{4}{3}\, r_\mathrm{cl}$.

|           | 3.1mm dust line | C+ 1   | C 1    | CO 1   |
| --------- | --------------- | ------ | ------ | ------ |
| intensity | 0.000398 K      | 61.1 K | 1.23 K | 5.35 K |

## 2. Accounting for absorption

The object here is to show the effect of absorption when there are overlapping clumps in a line-of-sight.

### Model solution

Here we consider a larger voxel with larger mass, but we still consider a simple model with one clump mass. What we can show, for example, is how different clump sets can be created for the same voxel. The mass of dense clump component in the model is the same ($10 \ M_\odot$), but we are now modelling a larger ensemble mass of $10^4 M_\odot$, a high density of $10^6 cm^{-3}$, and FUV radiation $10^4 \chi_0$. The additional clump set we wish to include is the diffuse interclump component. We model this with $10^{-2} M_\odot$ clumps, ensemble mass $10^2 M_\odot$, density $1911 cm^{-3}$, and FUV radiation $1 \chi_0$.

In [ ]:
from kosmatau3d import models
import numpy as np

np.set_printoptions(precision=8)

# Use these to change the voxel properties, which is necessary in the default mode.
parameters = {
              # Grid parameters
              'clump_tau_grid_file': 'clump_tau_LineCenter.dat',
              'clump_tb_grid_file': 'clump_Tmb_LineCenter.dat',
              'clump_taufuv_grid_file': 'RhoMassAFUV.dat',
              'clump_column_density_file': 'clumpMeanCols.dat',
              'clump_temperature_file': 'clumpTemperatures_filled.dat',
              'interclump_tau_grid_file': 'interclumpTauLineCenter.dat',
              'interclump_tb_grid_file': 'interclumpTmbLineCenter.dat',
              'interclump_dust_tau_grid_file': 'interclumpDustTau.dat',
              'interclump_dust_tb_grid_file': 'interclumpDustSED.dat',
              'interclump_taufuv_grid_file': 'interclumpTauFUV.dat',
              'interclump_column_density_file': 'interclumpMeanCols.dat',
              'interclump_temperature_file': 'interclumpTemperatures_filled.dat',

              # Model parameters
              'voxel_size' : 1,
              'transitions' : ['all'],
              'dust' : 'PAH',
              'clump_mass_range' : [[1],[-2]],
              'clump_mass_number' : [1,1],
              'clump_n_max' : [1,100],
              'velocity_range' : [-10, 10],
              'velocity_number' : 201,

              # Voxel properties
              'velocity' : 0.,
              'ensemble_dispersion' : [1,1],
              'ensemble_mass' : [1e5,1e2],
              'ensemble_density' : [1e6,1911],
              'fuv' : [1e4,1]
              }

vox = models.Voxel()
vox.set_properties(**parameters)
vox.calculate_emission(verbose=True)

We must first ensure the voxel properties are correctly modelled.

In [ ]:
models.masspoints.clump_log_density

In [ ]:
vox.get_ensemble_mass()

In [ ]:
vox.get_model_mass()

In [ ]:
vox.get_volume_filling_factor()

Here our model is filling too much of the voxel. We can keep approximately the same proportions of the two clumps sets by setting `volumeFactor`$=[0.66, 0.34]$.

In [ ]:
from kosmatau3d import models
import numpy as np

# Use these to change the voxel properties, which is necessary in the default mode.
parameters = {
              # Grid parameters
              'clump_tau_grid_file': 'clump_tau_LineCenter.dat',
              'clump_tb_grid_file': 'clump_Tmb_LineCenter.dat',
              'clump_taufuv_grid_file': 'RhoMassAFUV.dat',
              'clump_column_density_file': 'clumpMeanCols.dat',
              'clump_temperature_file': 'clumpTemperatures_filled.dat',
              'interclump_tau_grid_file': 'interclumpTauLineCenter.dat',
              'interclump_tb_grid_file': 'interclumpTmbLineCenter.dat',
              'interclump_dust_tau_grid_file': 'interclumpDustTau.dat',
              'interclump_dust_tb_grid_file': 'interclumpDustSED.dat',
              'interclump_taufuv_grid_file': 'interclumpTauFUV.dat',
              'interclump_column_density_file': 'interclumpMeanCols.dat',
              'interclump_temperature_file': 'interclumpTemperatures_filled.dat',

              # Model parameters
              'voxel_size' : 1,
              'transitions' : ['all'],
              'dust' : 'PAH',
              'clump_mass_range' : [[1],[-2]],
              'clump_mass_number' : [1,1],
              'clump_n_max' : [1,100],
              'velocity_range' : [-10, 10],
              'velocity_number' : 201,

              # Voxel properties
              'velocity' : 0.,
              'ensemble_dispersion' : [1,1],
              'volume_factor' : [0.66, 0.34],
              'ensemble_mass' : [1e5,1e2],
              'ensemble_density' : [1e6,1911],
              'fuv' : [1e4,1],
              }

vox = models.Voxel()
vox.set_properties(**parameters)
vox.calculate_emission(verbose=True)

Again we will ensure the voxel properties are correctly modelled. We can also verify that the voxel mean ensemble density has been adjusted.

In [ ]:
vox.get_ensemble_mass()

In [ ]:
vox.get_model_mass()

In [ ]:
vox.get_volume_filling_factor()

In [ ]:
vox.get_density()

Thes now verifies that the code alters the density rather than the mass when using the `volumeFactor` kwarg. Now we can get some information needed for the theoretical solution, which will be calculated in the next subsection.

In [ ]:
models.masspoints.clump_radius #in pc

In [ ]:
transitions = ['C+ 1', 'C 1', 'CO 1']
i_trans = [np.where(np.asarray(models.species.clump_transitions)==trans)[0][0] for trans in transitions]

models.masspoints.clump_species_intensity[0][:, i_trans]

In [ ]:
i_dust = models.constants.dust_names[models.constants.n_dust]=='3.1mm'

models.masspoints.clump_dust_intensity[0][:, i_dust]

In [ ]:
models.masspoints.clump_species_optical_depth[0][:, i_trans]

In [ ]:
models.masspoints.clump_dust_optical_depth[0][:, i_dust]

Finally, the intensity and optical depth of the single-voxel model is shown.

In [ ]:
%matplotlib inline

vox.plot_transition(transition=['C+ 1', 'C 1', 'CO 1'], quantity='intensity', title='', total=False, logscale=False)

In [ ]:
%matplotlib inline

vox.plot_transition(transition=['C+ 1', 'C 1', 'CO 1'], quantity='intensity', title='', total=True, logscale=False)

In [ ]:
vox.get_taufuv()

In [ ]:
vox.get_species_intensity(total=False)[0][100, i_trans]

In [ ]:
vox.get_dust_intensity(total=False)[0][100, i_dust]

In [ ]:
vox.get_species_emissivity(total=False)[0][100, i_trans]

In [ ]:
vox.get_dust_emissivity(total=False)[0][100, i_dust]

In [ ]:
vox.get_species_absorption(total=False)[0][100, i_trans]

In [ ]:
vox.get_dust_absorption(total=False)[0][100, i_dust]

In [ ]:
calc_trans = ['C+ 1', 'C 1', 'CO 1']
i_calc = [np.where(np.asarray(models.species.clump_transitions)==trans)[0][0] for trans in calc_trans]

for i in range(2):
  
    print('\n\nC L U M P   S E T   {}\n'.format(i))
    
    eps_CII, eps_C, eps_CO = vox.get_species_emissivity(total=False)[i][:, i_calc].max(0)
    kap_CII, kap_C, kap_CO = vox.get_species_absorption(total=False)[i][:, i_calc].max(0)

    #compute I via radiative transfer; in K
    I_CII = eps_CII/kap_CII * (1-np.exp(-kap_CII*models.constants.voxel_size))
    I_C = eps_C/kap_C * (1-np.exp(-kap_C*models.constants.voxel_size))
    I_CO = eps_CO/kap_CO * (1-np.exp(-kap_CO*models.constants.voxel_size))

    print('The peak brightness temperature of the first transition of C+ is {:.6f}'.format(I_CII))
    print('The peak brightness temperature of the first transition of C is {:.6f}'.format(I_C))
    print('The peak brightness temperature of the first transition of CO is {:.6f}'.format(I_CO))

### Analytic solution

#### clump set 1

We now need to compare this result to the expected analytical intensity. Having a large number of clumps in the model means the probability of seeing a certain number of clumps is peaked in a narrow range. (ie. We can use this value to approximate the optical depth.) The probability mass distribution peaks at ~19, as seen below.

In [ ]:
import numpy as np
from scipy.stats import binom
import matplotlib.pyplot as plt

rcl = models.masspoints.clump_radius[0][0,0]
p = np.pi*rcl**2

k = np.arange(0,50)
prob = binom.pmf(k,1e4, p)
plt.plot(k, prob)
plt.plot([19,19], [0,0.09], ls='--', c='xkcd:bright red')
plt.text(21,0,'19',color='xkcd:bright red', fontsize=16)
plt.xlabel('k', fontsize=20)
plt.ylabel(r'$\mathcal{P}$', fontsize=20) ;

We can directly calculate the intensity that we would expect in such a voxel. With more than one clump in our line-of-sight, we will also need to account for the total amount of absorption in the voxel. We will need to calculate $\epsilon_{\nu,cl}$ and $\kappa_{\nu,cl}$ for the clump, which is constant.

(8)
$$
\epsilon_{\nu, cl} = \frac{I_{\nu,cl}}{2R_{cl}}
$$

(9)
$$
\kappa_{\nu, cl} = \frac{\tau_{\nu,cl}}{2R_{cl}}
$$

Now we can use these values to integrate the radiative equation:

(10)
$$
I_\nu = e^{-\int_0^{\Delta s} \kappa_{\nu,cl} ds} \left( \int_0^{\Delta s} \epsilon_{\nu,cl} e^{\int_0^{\Delta s} \kappa_{\nu,cl} s'} ds' + I_{bg} \right)
$$

Since the emission and absorption coefficients $\left( \epsilon_{\nu,cl}, \kappa_{\nu,cl} \right)$ are valid just in each clump, integration over $\mathcal{N}$ clumps with $I_{bg}=0$ gives,


(11)
$$
I_\nu = \frac{\epsilon_{\nu,cl}}{\kappa_{\nu,cl}} \left( 1 - e^{-\kappa_{\nu,cl} (2\mathcal{N}R_{cl})} \right)
$$

We can now analytically calculate what is the intensity of a given transition if we have 66 clumps in our line-of-sight.

In [ ]:
# Define how many clumps are in a line-of-sight
nClumps = 19
# Get the clump radius
rcl = models.masspoints.clump_radius[0][0,0]
# Specify transitions
calc_trans = ['C+ 1', 'C 1', 'CO 1']
i_calc = [np.where(np.asarray(models.species.clump_transitions)==trans)[0][0] for trans in calc_trans]
# get the clump intensity and optical depth for the required transitions
icl = models.masspoints.clump_species_intensity[0][0, i_calc]
tcl = models.masspoints.clump_species_optical_depth[0][0, i_calc]

# Uncomment these if you would like to compare to the individual clump intensities
print('Clump absorption coefficient:')
for i,I in enumerate(tcl/2/rcl):
    print('{:>10}: {:.6}'.format(calc_trans[i], I))
print('\n')
print('Clump intensity:')
for i,I in enumerate(icl/tcl*(1-np.exp(-tcl))):
    print('{:>10}: {:.6} K'.format(calc_trans[i], I))
print('\n')

# Initialise the line intensities
I = np.zeros(icl.size)

# Integrate over the number of clumps along a line-of-sight
# for i in range(nClumps):
#   I = np.exp(-tcl) * ( icl/tcl*(np.exp(tcl)-1) + I )
  
# Integrate in an alternate simplified equation
I = icl/tcl * (1-np.exp(-nClumps*tcl))

for i,transition in enumerate(calc_trans):
    print('The brightness temperature of the {:^10} transition is {:.6} K'.format(transition, I[i]))

These values are very similar to the results of the single-voxel model. The accurracy of this result is due to the large number of clumps in the voxel, giving a very specific number of clump in a line-of-sight. The difference between this result and the intensity of a single clump is larger for transitions with lower optical depth. That is how the $\mathrm{CO} \ 1 \rightarrow 0$ and $\mathrm{CII} \ 1 \rightarrow 0$ transitions get larger than that of a single clump even though they are saturated; the background intensity has a larger effect.

There is also a difference between this analytic result and the one obtained with the single-voxel model in the $\mathrm{CI} \ 1 \rightarrow 0$ transition. The issue here is the ensemble velocity dispersion in the single-voxel model. We gave it a velocity dispersion of $1 \frac{km}{s}$—very close to the intrinsic clump velocity dispersion of $0.71 \frac{km}{s}$—but even this is enough to ensure the $10^4$ clumps are actually spread over the ensemble velocities. In this way, it reduces the peak intensity of the voxel. The reason it only is seen in the $\mathrm{CI} \ 1 \rightarrow 0$ transition is that it is not suffering from any saturation effect (you can see this by the lack of a plateau feature in the velocity-intensity plot seen earlier).

#### clump set 2

In [ ]:
import numpy as np
from scipy.stats import binom
import matplotlib.pyplot as plt

rcl = models.masspoints.clumpRadius[1][0,0]
p = np.pi*rcl**2

k = np.arange(0,50)
prob = binom.pmf(k,1e4, p)
plt.plot(k, prob)
plt.plot([12,12], [0,0.11], ls='--', c='xkcd:bright red')
plt.text(14,0,'12',color='xkcd:bright red', fontsize=16)
plt.xlabel('k', fontsize=20)
plt.ylabel(r'$\mathcal{P}$', fontsize=20) ;

We can directly calculate the intensity that we would expect in such a voxel. With more than one clump in our line-of-sight, we will also need to account for the total amount of absorption in the voxel. We will need to calculate $\epsilon_{\nu,cl}$ and $\kappa_{\nu,cl}$ for the clump, which is constant.

(8)
$$
\epsilon_{\nu, cl} = \frac{I_{\nu,cl}}{2R_{cl}}
$$

(9)
$$
\kappa_{\nu, cl} = \frac{\tau_{\nu,cl}}{2R_{cl}}
$$

Now we can use these values to integrate the radiative equation:

(10)
$$
I_\nu = e^{-\int_0^{\Delta s} \kappa_{\nu,cl} ds} \left( \int_0^{\Delta s} \epsilon_{\nu,cl} e^{\int_0^{\Delta s} \kappa_{\nu,cl} s'} ds' + I_{bg} \right)
$$

Since the emission and absorption coefficients $\left( \epsilon_{\nu,cl}, \kappa_{\nu,cl} \right)$ are valid just in each clump, integration over $\mathcal{N}$ clumps with $I_{bg}=0$ gives,


(11)
$$
I_\nu = \frac{\epsilon_{\nu,cl}}{\kappa_{\nu,cl}} \left( 1 - e^{-\kappa_{\nu,cl} (2\mathcal{N}R_{cl})} \right)
$$

We can now analytically calculate what is the intensity of a given transition if we have 12 clumps in our line-of-sight.

In [ ]:
# Define how many clumps are in a line-of-sight
nClumps = 12
# Get the clump radius
rcl = models.masspoints.clumpRadius[0][0,0]
# Specify transitions
molecules = ['dust 3.1mm', 'C+ 1', 'C 1', 'CO 1']
# get the clump intensity and optical depth for the required transitions
icl = models.masspoints.clumpIntensity[1][0,models.species.speciesIndeces(molecules)]
tcl = models.masspoints.clumpOpticalDepth[1][0,models.species.speciesIndeces(molecules)]

# Uncomment these to view the clump intensities
print('Clump absorption coefficient:')
for i,I in enumerate(tcl/2/rcl):
    print('{:>10}: {:.6}'.format(molecules[i], I))
print('\n')
print('Clump intensity:')
for i,I in enumerate(icl/tcl*(1-np.exp(-tcl))):
    print('{:>10}: {:.6} K'.format(molecules[i], I))
print('\n')

# Initialise the line intensities
I = np.zeros(icl.size)

# Integrate over the number of clumps along a line-of-sight
# for i in range(nClumps):
#   I = np.exp(-tcl) * ( icl/tcl*(np.exp(tcl)-1) + I )
  
# Integrate in an alternate simplified equation
I = icl/tcl * (1-np.exp(-nClumps*tcl))

for i,transition in enumerate(molecules):
    print('The brightness temperature of the {:^10} transition is {:.6} K'.format(transition, I[i]))

Like in the previous clump set, these values are very similar to the results of the single-voxel model. The accurracy of this result is due to the large number of clumps in the voxel, giving a very specific number of clumps in a line-of-sight.

This time the difference between the analytic result and the one obtained with the single-voxel model is due primarily to the ensemble velocity dispersion. We gave it a velocity dispersion of $1 \frac{km}{s}$—very close to the intrinsic clump velocity dispersion of $0.71 \frac{km}{s}$—but even this is enough to ensure the $10^4$ clumps are actually spread over the ensemble velocities. In this way, it reduces the peak intensity of the voxel. It is evident in the $\mathrm{CI} \ 1 \rightarrow 0$ transition since it is not suffering from a saturation effect (you can see this by the lack of a plateau feature in the velocity-intensity plot above earlier).

## 3. Changing the velocity dispersion in the voxel

Here we will consider two voxel with similar properties except for the ensemble dispersion. Thus we will examine the how the velocity dispersion affects the resulting intensity.

In [ ]:
from kosmatau3d import models
import numpy as np

np.set_printoptions(precision=8)

# Use these to change the voxel properties, which is necessary in the default mode.
parameters = {
              # Grid parameters
              'clump_tau_grid_file': 'clump_tau_LineCenter.dat',
              'clump_tb_grid_file': 'clump_Tmb_LineCenter.dat',
              'clump_taufuv_grid_file': 'RhoMassAFUV.dat',
              'clump_column_density_file': 'clumpMeanCols.dat',
              'clump_temperature_file': 'clumpTemperatures_filled.dat',
              'interclump_tau_grid_file': 'interclumpTauLineCenter.dat',
              'interclump_tb_grid_file': 'interclumpTmbLineCenter.dat',
              'interclump_dust_tau_grid_file': 'interclumpDustTau.dat',
              'interclump_dust_tb_grid_file': 'interclumpDustSED.dat',
              'interclump_taufuv_grid_file': 'interclumpTauFUV.dat',
              'interclump_column_density_file': 'interclumpMeanCols.dat',
              'interclump_temperature_file': 'interclumpTemperatures_filled.dat',

              # Model parameters
              'voxel_size' : 1,
              'transitions' : 'all',
              'dust' : 'PAH',
              'clump_mass_range' : [[1]],
              'clump_mass_number' : [1],
              'clump_n_max' : [1],
              'velocity_range' : [-10, 10],
              'velocity_number' : 201,

              # Voxel properties
              'velocity' : 0.,
              'ensemble_dispersion' : 1.5,
              'ensemble_mass' : 1e4,
              'ensemble_density' : 1e6,
              'fuv' : 1e4
              }

vox = models.Voxel()
vox.set_properties(**parameters)
vox.calculate_emission(verbose=True)
vox.plot_transition(transition=['C+ 1', 'C 1', 'CO 1'])

In [ ]:
vox.get_volume_filling_factor()

In [ ]:
from kosmatau3d import models
import numpy as np

np.set_printoptions(precision=8)

# Use these to change the voxel properties, which is necessary in the default mode.
parameters = {
              # Grid parameters
              'clump_tau_grid_file': 'clump_tau_LineCenter.dat',
              'clump_tb_grid_file': 'clump_Tmb_LineCenter.dat',
              'clump_taufuv_grid_file': 'RhoMassAFUV.dat',
              'clump_column_density_file': 'clumpMeanCols.dat',
              'clump_temperature_file': 'clumpTemperatures_filled.dat',
              'interclump_tau_grid_file': 'interclumpTauLineCenter.dat',
              'interclump_tb_grid_file': 'interclumpTmbLineCenter.dat',
              'interclump_dust_tau_grid_file': 'interclumpDustTau.dat',
              'interclump_dust_tb_grid_file': 'interclumpDustSED.dat',
              'interclump_taufuv_grid_file': 'interclumpTauFUV.dat',
              'interclump_column_density_file': 'interclumpMeanCols.dat',
              'interclump_temperature_file': 'interclumpTemperatures_filled.dat',

              # Model parameters
              'voxel_size' : 1,
              'transitions' : 'all',
              'dust' : 'PAH',
              'clump_mass_range' : [[1]],
              'clump_mass_number' : [1],
              'clump_n_max' : [1],
              'velocity_range' : [-10, 10],
              'velocity_number' : 201,

              # Voxel properties
              'velocity' : 0.,
              'ensemble_dispersion' : 3,
              'ensemble_mass' : 1e4,
              'ensemble_density' : 1e6,
              'fuv' : 1e4
              }

vox = models.Voxel()
vox.set_properties(**parameters)
vox.calculate_emission(verbose=True)
vox.plot_transition(transition=['C+ 1', 'C 1', 'CO 1'])

In [ ]:
vox.get_volume_filling_factor()

This clearly shows the effect of increasing the ensemble velocity dispersion. While the first model was starting to show saturation effects in $\mathrm{CII}$ and $\mathrm{CO}$, these are not present in the second model with a higher ensemble velocity dispersion.

## 4. Resizing the voxel

### Small clumps

Here we consider a voxel with larger mass and more observed velocities, but we still consider a simple model with one clump mass. The mass clump this time is much smaller ($10^{-2} \ M_\odot$). This is the typical clump size used to model the interclump medium. Andree-Labsch (2017) models the interclump medium by forcing $n_\mathrm{max}=100$ (this is called `clumpNmax` in the single-voxel model). As you might have seen, specifying `clumpNmax` will normalise the number of clumps in the model to that value by scaling the voxel size. Using a value of None will not use this feature. We can model $10^2 M_\odot$ in the voxel, but vary $n_\mathrm{max} \in (100, 1, \mathrm{None})$. This example will be used to examine the validity of this normalisation and rescaling of the voxel.

In [ ]:
from kosmatau3d import models
import numpy as np

# Use these to change the voxel properties, which is necessary in the default mode.
parameters = {
              # Grid parameters
              'clump_tau_grid_file': 'clump_tau_LineCenter.dat',
              'clump_tb_grid_file': 'clump_Tmb_LineCenter.dat',
              'clump_taufuv_grid_file': 'RhoMassAFUV.dat',
              'clump_column_density_file': 'clumpMeanCols.dat',
              'clump_temperature_file': 'clumpTemperatures_filled.dat',
              'interclump_tau_grid_file': 'interclumpTauLineCenter.dat',
              'interclump_tb_grid_file': 'interclumpTmbLineCenter.dat',
              'interclump_dust_tau_grid_file': 'interclumpDustTau.dat',
              'interclump_dust_tb_grid_file': 'interclumpDustSED.dat',
              'interclump_taufuv_grid_file': 'interclumpTauFUV.dat',
              'interclump_column_density_file': 'interclumpMeanCols.dat',
              'interclump_temperature_file': 'interclumpTemperatures_filled.dat',

              # Model parameters
              'voxel_size' : 1,
              'transitions' : 'all',
              'dust' : 'PAH',
              'clump_mass_range' : [[-2]],
              'clump_mass_number' : [1],
              'clump_n_max' : [100],
              'velocity_range' : [-10, 10],
              'velocity_number' : 201,

              # Voxel properties
              'velocity' : 0.,
              'ensemble_dispersion' : 3,
              'ensemble_mass' : 1e2,
              'ensemble_density' : 1e6,
              'fuv' : 1e4
              }

vox = models.Voxel()
vox.set_properties(**parameters)
vox.calculate_emission(verbose=True)

transitions = ['C+ 1', 'C 1', 'CO 1']
i_trans = [np.where(np.asarray(models.species.clump_transitions)==trans)[0][0] for trans in transitions]
print('Peak intensities for ', transitions)
print(vox.get_species_intensity()[0][:, i_trans].max(0))
# vox.plotMolecule(molecule=molecules)

In [ ]:
vox.get_volume_filling_factor()

In [ ]:
vox.get_volume_filling_factor()

In [ ]:
models.ensemble.print_ensembleparameters()

In [ ]:
from kosmatau3d import models
import numpy as np

# Use these to change the voxel properties, which is necessary in the default mode.
parameters = {
              # Grid parameters
              'clump_tau_grid_file': 'clump_tau_LineCenter.dat',
              'clump_tb_grid_file': 'clump_Tmb_LineCenter.dat',
              'clump_taufuv_grid_file': 'RhoMassAFUV.dat',
              'clump_column_density_file': 'clumpMeanCols.dat',
              'clump_temperature_file': 'clumpTemperatures_filled.dat',
              'interclump_tau_grid_file': 'interclumpTauLineCenter.dat',
              'interclump_tb_grid_file': 'interclumpTmbLineCenter.dat',
              'interclump_dust_tau_grid_file': 'interclumpDustTau.dat',
              'interclump_dust_tb_grid_file': 'interclumpDustSED.dat',
              'interclump_taufuv_grid_file': 'interclumpTauFUV.dat',
              'interclump_column_density_file': 'interclumpMeanCols.dat',
              'interclump_temperature_file': 'interclumpTemperatures_filled.dat',

              # Model parameters
              'voxel_size' : 1,
              'transitions' : 'all',
              'dust' : 'PAH',
              'clump_mass_range' : [[-2]],
              'clump_mass_number' : [1],
              'clump_n_max' : [1],
              'velocity_range' : [-10, 10],
              'velocity_number' : 201,

              # Voxel properties
              'velocity' : 0.,
              'ensemble_dispersion' : 3,
              'ensemble_mass' : 1e2,
              'ensemble_density' : 1e6,
              'fuv' : 1e4,
              }

vox = models.Voxel()
vox.set_properties(**parameters)
vox.calculate_emission(verbose=True)

transitions = ['C+ 1', 'C 1', 'CO 1']
i_trans = [np.where(np.asarray(models.species.clump_transitions)==trans)[0][0] for trans in transitions]
print('Peak intensities for ', transitions)
print(vox.get_species_intensity()[0][:, i_trans].max(0))
# vox.plotMolecule(molecule=molecules)

In [ ]:
vox.get_volume_filling_factor()

In [ ]:
models.ensemble.print_ensembleparameters()

In [ ]:
from kosmatau3d import models
import numpy as np

# Use these to change the voxel properties, which is necessary in the default mode.
parameters = {
              # Grid parameters
              'clump_tau_grid_file': 'clump_tau_LineCenter.dat',
              'clump_tb_grid_file': 'clump_Tmb_LineCenter.dat',
              'clump_taufuv_grid_file': 'RhoMassAFUV.dat',
              'clump_column_density_file': 'clumpMeanCols.dat',
              'clump_temperature_file': 'clumpTemperatures_filled.dat',
              'interclump_tau_grid_file': 'interclumpTauLineCenter.dat',
              'interclump_tb_grid_file': 'interclumpTmbLineCenter.dat',
              'interclump_dust_tau_grid_file': 'interclumpDustTau.dat',
              'interclump_dust_tb_grid_file': 'interclumpDustSED.dat',
              'interclump_taufuv_grid_file': 'interclumpTauFUV.dat',
              'interclump_column_density_file': 'interclumpMeanCols.dat',
              'interclump_temperature_file': 'interclumpTemperatures_filled.dat',

              # Model parameters
              'voxel_size' : 1,
              'transitions' : 'all',
              'dust' : 'PAH',
              'clump_mass_range' : [[-2]],
              'clump_mass_number' : [1],
              'clump_n_max' : [None],
              'velocity_range' : [-10, 10],
              'velocity_number' : 201,

              # Voxel properties
              'velocity' : 0.,
              'ensemble_dispersion' : 3,
              'ensemble_mass' : 1e2,
              'ensemble_density' : 1e6,
              'fuv' : 1e4
              }

vox = models.Voxel()
vox.set_properties(**parameters)
vox.calculate_emission(verbose=True)

transitions = ['C+ 1', 'C 1', 'CO 1']
i_trans = [np.where(np.asarray(models.species.clump_transitions)==trans)[0][0] for trans in transitions]
print('Peak intensities for ', transitions)
print(vox.get_species_intensity()[0][:, i_trans].max(0))
# vox.plotMolecule(molecule=molecules)

In [ ]:
vox.get_volume_filling_factor()

In [ ]:
models.ensemble.print_ensembleparameters()

As we can see here, changing $\mathrm{n}_{max}$ has little effect on the resulting intensity of the voxel $(< 5 \%)$. The main motivation for specifying $\mathrm{n}_{max}$ is a computational one: it limits the number of clump combinations calculated and thus the computational footprint of the model. Approximating the voxel with $\mathrm{n}_{max}=100$ is not useful in this case, since the resulting intensities are higher than the un-normalised calculations.

### Large clumps

One reason the test using $10^{-2} M_\odot$ did not show much of a difference using different $\mathrm{n}_{max}$ is that there is not much absorption in such a small clump mass. We can thus repeat this experiment for a more massive clump of $10 M_\odot$. In this case we use an ensemble mass of $10^3 M_\odot$ to ensure the voxel still has enough clumps to suffer from self-absorption but not so many to be completely self-absorbed (seen by a plateau in the velocity-intensity plot).

In [ ]:
from kosmatau3d import models
import numpy as np

# Use these to change the voxel properties, which is necessary in the default mode.
parameters = {
              # Grid parameters
              'clump_tau_grid_file': 'clump_tau_LineCenter.dat',
              'clump_tb_grid_file': 'clump_Tmb_LineCenter.dat',
              'clump_taufuv_grid_file': 'RhoMassAFUV.dat',
              'clump_column_density_file': 'clumpMeanCols.dat',
              'clump_temperature_file': 'clumpTemperatures_filled.dat',
              'interclump_tau_grid_file': 'interclumpTauLineCenter.dat',
              'interclump_tb_grid_file': 'interclumpTmbLineCenter.dat',
              'interclump_dust_tau_grid_file': 'interclumpDustTau.dat',
              'interclump_dust_tb_grid_file': 'interclumpDustSED.dat',
              'interclump_taufuv_grid_file': 'interclumpTauFUV.dat',
              'interclump_column_density_file': 'interclumpMeanCols.dat',
              'interclump_temperature_file': 'interclumpTemperatures_filled.dat',

              # Model parameters
              'voxel_size' : 1,
              'transitions' : 'all',
              'dust' : 'PAH',
              'clump_mass_range' : [[1]],
              'clump_mass_number' : [1],
              'clump_n_max' : [1],
              'velocity_range' : [-10, 10],
              'velocity_number' : 201,

              # Voxel properties
              'velocity' : 0.,
              'ensemble_dispersion' : 1,
              'ensemble_mass' : 1e3,
              'ensemble_density' : 1e6,
              'fuv' : 1e4
              }

vox = models.Voxel()
vox.set_properties(**parameters)
vox.calculate_emission(verbose=True)

transitions = ['C+ 1', 'C 1', 'CO 1']
i_trans = [np.where(np.asarray(models.species.clump_transitions)==trans)[0][0] for trans in transitions]
print('Peak intensities for ', transitions)
print(vox.get_species_intensity()[0][:, i_trans].max(0))
# vox.plotMolecule(molecule=molecules)

In [ ]:
vox.get_volume_filling_factor()

In [ ]:
models.ensemble.print_ensembleparameters()

In [ ]:
from kosmatau3d import models
import numpy as np

# Use these to change the voxel properties, which is necessary in the default mode.
parameters = {
              # Grid parameters
              'clump_tau_grid_file': 'clump_tau_LineCenter.dat',
              'clump_tb_grid_file': 'clump_Tmb_LineCenter.dat',
              'clump_taufuv_grid_file': 'RhoMassAFUV.dat',
              'clump_column_density_file': 'clumpMeanCols.dat',
              'clump_temperature_file': 'clumpTemperatures_filled.dat',
              'interclump_tau_grid_file': 'interclumpTauLineCenter.dat',
              'interclump_tb_grid_file': 'interclumpTmbLineCenter.dat',
              'interclump_dust_tau_grid_file': 'interclumpDustTau.dat',
              'interclump_dust_tb_grid_file': 'interclumpDustSED.dat',
              'interclump_taufuv_grid_file': 'interclumpTauFUV.dat',
              'interclump_column_density_file': 'interclumpMeanCols.dat',
              'interclump_temperature_file': 'interclumpTemperatures_filled.dat',

              # Model parameters
              'voxel_size' : 1,
              'transitions' : 'all',
              'dust' : 'PAH',
              'clump_mass_range' : [[1]],
              'clump_mass_number' : [1],
              'clump_n_max' : [3],
              'velocity_range' : [-10, 10],
              'velocity_number' : 201,

              # Voxel properties
              'velocity' : 0.,
              'ensemble_dispersion' : 1,
              'ensemble_mass' : 1e3,
              'ensemble_density' : 1e6,
              'fuv' : 1e4
              }

vox = models.Voxel()
vox.set_properties(**parameters)
vox.calculate_emission(verbose=True)

transitions = ['C+ 1', 'C 1', 'CO 1']
i_trans = [np.where(np.asarray(models.species.clump_transitions)==trans)[0][0] for trans in transitions]
print('Peak intensities for ', transitions)
print(vox.get_species_intensity()[0][:, i_trans].max(0))
# vox.plotMolecule(molecule=molecules)

In [ ]:
vox.get_volume_filling_factor()

In [ ]:
models.ensemble.print_ensembleparameters()

In [ ]:
from kosmatau3d import models
import numpy as np

# Use these to change the voxel properties, which is necessary in the default mode.
parameters = {
              # Grid parameters
              'clump_tau_grid_file': 'clump_tau_LineCenter.dat',
              'clump_tb_grid_file': 'clump_Tmb_LineCenter.dat',
              'clump_taufuv_grid_file': 'RhoMassAFUV.dat',
              'clump_column_density_file': 'clumpMeanCols.dat',
              'clump_temperature_file': 'clumpTemperatures_filled.dat',
              'interclump_tau_grid_file': 'interclumpTauLineCenter.dat',
              'interclump_tb_grid_file': 'interclumpTmbLineCenter.dat',
              'interclump_dust_tau_grid_file': 'interclumpDustTau.dat',
              'interclump_dust_tb_grid_file': 'interclumpDustSED.dat',
              'interclump_taufuv_grid_file': 'interclumpTauFUV.dat',
              'interclump_column_density_file': 'interclumpMeanCols.dat',
              'interclump_temperature_file': 'interclumpTemperatures_filled.dat',

              # Model parameters
              'voxel_size' : 1,
              'transitions' : 'all',
              'dust' : 'PAH',
              'clump_mass_range' : [[1]],
              'clump_mass_number' : [1],
              'clump_n_max' : [None],
              'velocity_range' : [-10, 10],
              'velocity_number' : 201,

              # Voxel properties
              'velocity' : 0.,
              'ensemble_dispersion' : 1,
              'ensemble_mass' : 1e3,
              'ensemble_density' : 1e6,
              'fuv' : 1e4
              }

vox = models.Voxel()
vox.set_properties(**parameters)
vox.calculate_emission(verbose=True)

transitions = ['C+ 1', 'C 1', 'CO 1']
i_trans = [np.where(np.asarray(models.species.clump_transitions)==trans)[0][0] for trans in transitions]
print('Peak intensities for ', molecules)
print(vox.get_species_intensity()[0][:, i_trans].max(0))
# vox.plotMolecule(molecule=molecules)

In [ ]:
vox.get_volume_filling_factor()

In [ ]:
models.ensemble.print_ensembleparameters()

The difference here can approach $5 \%$. This difference, while not much, should be kept in mind when using the single-voxel model. This error is also most evident in models with enough clumps for self-absorption, but not so many that a saturation plateau appears in the velocity-intensity plot. In cases that feature a plateau, this error is mitigated.

## 5. Dense voxel with multiple clump masses

Having shown the functionality of using one clump in the model, we now repeat the previous example using clump masses $0.1$, $1$, and $10 \ M_\odot$ in the model. What we will see is that there is a similar intensity and optical depth for clump of different sizes if one changes the mass accordingly. The probability of having a clump in a line-of-sight depends on the voxel surface area, which depends on $\Delta s^2$. The mass density, however, is proportional to $\Delta s^{-3}$. We will now examine the effect of changing the voxel size by a factor of $10$ and the ensemble mass by a factor of $100$.

In [ ]:
from kosmatau3d import models
import numpy as np

# Use these to change the voxel properties, which is necessary in the default mode.
parameters = {
              # Grid parameters
              'clump_tau_grid_file': 'clump_tau_LineCenter.dat',
              'clump_tb_grid_file': 'clump_Tmb_LineCenter.dat',
              'clump_taufuv_grid_file': 'RhoMassAFUV.dat',
              'clump_column_density_file': 'clumpMeanCols.dat',
              'clump_temperature_file': 'clumpTemperatures_filled.dat',
              'interclump_tau_grid_file': 'interclumpTauLineCenter.dat',
              'interclump_tb_grid_file': 'interclumpTmbLineCenter.dat',
              'interclump_dust_tau_grid_file': 'interclumpDustTau.dat',
              'interclump_dust_tb_grid_file': 'interclumpDustSED.dat',
              'interclump_taufuv_grid_file': 'interclumpTauFUV.dat',
              'interclump_column_density_file': 'interclumpMeanCols.dat',
              'interclump_temperature_file': 'interclumpTemperatures_filled.dat',

              # Model parameters
              'voxel_size' : 0.1,
              'transitions' : 'all',
              'dust' : 'PAH',
              'clump_mass_range' : [[0, 2]],
              'clump_mass_number' : [3],
              'clump_n_max' : [1],
              'velocity_range' : [-10, 10],
              'velocity_number' : 201,

              # Voxel properties
              'velocity' : 0.,
              'ensemble_dispersion' : [2],
              'ensemble_mass' : [1e1],
              'ensemble_density' : [1e6],
              'fuv' : 1e4
              }

vox = models.Voxel()
vox.set_properties(**parameters)
vox.calculate_emission(verbose=True)
print('\nVolume-filling factor:')
print(vox.get_volume_filling_factor())
print()
vox.plot_transition(transition=['C+ 1', 'C 1', 'CO 1', '13C+ 1', '13C 1', '13CO 1'], quantity='intensity', logscale=False)

In [ ]:
from kosmatau3d import models
import numpy as np

# Use these to change the voxel properties, which is necessary in the default mode.
parameters = {
              # Grid parameters
              'clump_tau_grid_file': 'clump_tau_LineCenter.dat',
              'clump_tb_grid_file': 'clump_Tmb_LineCenter.dat',
              'clump_taufuv_grid_file': 'RhoMassAFUV.dat',
              'clump_column_density_file': 'clumpMeanCols.dat',
              'clump_temperature_file': 'clumpTemperatures_filled.dat',
              'interclump_tau_grid_file': 'interclumpTauLineCenter.dat',
              'interclump_tb_grid_file': 'interclumpTmbLineCenter.dat',
              'interclump_dust_tau_grid_file': 'interclumpDustTau.dat',
              'interclump_dust_tb_grid_file': 'interclumpDustSED.dat',
              'interclump_taufuv_grid_file': 'interclumpTauFUV.dat',
              'interclump_column_density_file': 'interclumpMeanCols.dat',
              'interclump_temperature_file': 'interclumpTemperatures_filled.dat',

              # Model parameters
              'voxel_size' : 1,
              'transitions' : 'all',
              'dust' : 'PAH',
              'clump_mass_range' : [[0, 2]],
              'clump_mass_number' : [3],
              'clump_n_max' : [1],
              'velocity_range' : [-10, 10],
              'velocity_number' : 201,

              # Voxel properties
              'velocity' : 0.,
              'ensemble_dispersion' : [2],
              # 'volumeFactor' : [1.],
              'ensemble_mass' : [1e3],
              'ensemble_density' : [10**6],
              'fuv' : 1e4,
              }

vox = models.Voxel()
vox.set_properties(**parameters)
vox.calculate_emission()
print('\nVolume-filling factor:')
print(vox.get_volume_filling_factor())
print()
vox.plot_transition(transition=['C+ 1', 'C 1', 'CO 1', '13C+ 1', '13C 1', '13CO 1'], quantity='intensity', logscale=False)

Indeed the same intensity results are present in these two models. Since we do not change the model clumps, the voxel density, or FUV field, this should be expected by comparing the results of `ensemble.printEnsembleParameters()`. The volume-filling factor, however, changes by a factor of $10$, since we did not scale the mass with $\Delta s^3$.

In [ ]:
display(Latex("The surface densities $\mathrm{log_{10}}(n_\mathrm{s, cl})$ are "
              + f"${', '.join([f'{n:.3f}' for n in models.masspoints.clump_log_density[0][0]])}$ in the clumpy ensemble "
              + f"for clumps with masses ${', '.join([f'{10**m:.0f}' for m in models.constants.clump_log_mass[0][0]])}\, M_\odot$"))
display(Latex("This is results in a clump-averaged density of $\mathrm{log_{10}}(n_\mathrm{ens})\! =\! "
              + f"{np.log10(vox.get_density()[0]):.1f}$."))

In [ ]:
display(Latex("The number of clumps, $N_\mathrm{cl}$, in the ensemble are "
              f"${', '.join([f'{n:.0f}' for n in models.ensemble.clumpNj[0][0]])}$ "
              f"for clumps with masses ${', '.join([f'{10**m:.0f}' for m in models.constants.clump_log_mass[0][0]])}\, M_\odot$"))